In [1]:
from pathlib import Path
from summarynb import show
import pandas as pd
from IPython.display import display, Markdown

from malid.external.summarynb_extras import empty
from malid import config, logger
from malid.datamodels import (
    GeneLocus,
    TargetObsColumnEnum,
    SampleWeightStrategy,
    combine_classification_option_names,
)
from malid.train import train_metamodel
from malid.trained_model_wrappers import BlendingMetamodel

In [2]:
model_name_overall_repertoire_composition = (
    config.metamodel_base_model_names.model_name_overall_repertoire_composition
)
model_name_convergent_clustering = (
    config.metamodel_base_model_names.model_name_convergent_clustering
)
model_name_sequence_disease = (
    config.metamodel_base_model_names.model_name_sequence_disease
)
base_model_train_fold_name = "train_smaller"
metamodel_fold_label_train = "validation"

In [3]:
def summary(
    gene_locus: GeneLocus,
    target_obs_column: TargetObsColumnEnum,
    sample_weight_strategy: SampleWeightStrategy = SampleWeightStrategy.ISOTYPE_USAGE,
    show_base=True,
    show_metamodel=True,
    show_best_metamodel_only=False,
):
    display(
        Markdown(f"### {gene_locus}, {target_obs_column}, {sample_weight_strategy}")
    )
    try:
        if show_base:
            show(
                [
                    [
                        config.paths.repertoire_stats_classifier_output_dir
                        / gene_locus.name
                        / combine_classification_option_names(
                            target_obs_column=target_obs_column
                        )
                        / f"train_smaller_model.test_set_performance.{model_name_overall_repertoire_composition}.classification_report.txt",
                        config.paths.convergent_clusters_output_dir
                        / gene_locus.name
                        / combine_classification_option_names(
                            target_obs_column=target_obs_column
                        )
                        / f"train_smaller_model.test_set_performance.{model_name_convergent_clustering}.classification_report.txt",
                        config.paths.sequence_models_output_dir
                        / gene_locus.name
                        / "rollup_models"
                        / combine_classification_option_names(
                            target_obs_column=target_obs_column,
                            sample_weight_strategy=sample_weight_strategy,
                        )
                        / f"sequence_prediction_rollup.{model_name_sequence_disease}.train_smaller_model.report.txt",
                    ],
                    [
                        config.paths.repertoire_stats_classifier_output_dir
                        / gene_locus.name
                        / combine_classification_option_names(
                            target_obs_column=target_obs_column
                        )
                        / f"train_smaller_model.test_set_performance.{model_name_overall_repertoire_composition}.confusion_matrix.png",
                        config.paths.convergent_clusters_output_dir
                        / gene_locus.name
                        / combine_classification_option_names(
                            target_obs_column=target_obs_column
                        )
                        / f"train_smaller_model.test_set_performance.{model_name_convergent_clustering}.confusion_matrix.png",
                        config.paths.sequence_models_output_dir
                        / gene_locus.name
                        / "rollup_models"
                        / combine_classification_option_names(
                            target_obs_column=target_obs_column,
                            sample_weight_strategy=sample_weight_strategy,
                        )
                        / f"sequence_prediction_rollup.{model_name_sequence_disease}.train_smaller_model.confusion_matrix.png",
                    ],
                ],
                headers=[
                    f"model1 {model_name_overall_repertoire_composition}",
                    f"model2 {model_name_convergent_clustering}",
                    f"model3-rollup {model_name_sequence_disease}",
                ],
                max_width=400,
            )

            display(Markdown("---"))

        if show_metamodel:
            try:
                flavors = train_metamodel.get_metamodel_flavors(
                    gene_locus=gene_locus,
                    target_obs_column=target_obs_column,
                    fold_id=config.all_fold_ids[0],
                    base_model_train_fold_name=base_model_train_fold_name,
                )
            except Exception as err:
                logger.warning(
                    f"Failed to generate metamodel flavors for {gene_locus}, {target_obs_column}: {err}"
                )
                return
            for metamodel_flavor in flavors.keys():
                display(Markdown(f"#### Metamodel flavor {metamodel_flavor}"))
                _output_suffix = (
                    Path(gene_locus.name)
                    / target_obs_column.name
                    / metamodel_flavor
                    / f"{base_model_train_fold_name}_applied_to_{metamodel_fold_label_train}_model"
                )
                metamodel_output_prefix = (
                    config.paths.second_stage_blending_metamodel_output_dir
                    / _output_suffix
                )
                metamodel_highres_results_output_prefix = (
                    config.paths.high_res_outputs_dir / "metamodel" / _output_suffix
                )
                metamodel_stats = pd.read_csv(
                    f"{metamodel_output_prefix}.compare_model_scores.test_set_performance.tsv",
                    sep="\t",
                    index_col=0,
                )

                if metamodel_stats.shape[0] == 0:
                    logger.warning(f"Metamodel not run")
                    return

                for metamodel_name in (
                    [metamodel_stats.index[0]]
                    if show_best_metamodel_only
                    else [
                        "linearsvm_ovr",
                        "lasso_cv",
                        "ridge_cv",
                        "elasticnet_cv",
                        "xgboost",
                        "rf_multiclass",
                    ]
                ):
                    show(
                        [
                            [
                                f"{metamodel_output_prefix}.classification_report.test_set_performance.{metamodel_name}.txt",
                                f"{metamodel_highres_results_output_prefix}.errors_versus_difference_between_top_two_probabilities.test_set_performance.{metamodel_name}.with_abstention.vertical.png",
                                f"{metamodel_highres_results_output_prefix}.feature_importances.{metamodel_name}.raw_coefs.png"
                                if metamodel_name != "rf_multiclass"
                                else f"{metamodel_highres_results_output_prefix}.feature_importances.{metamodel_name}.all.png",
                                f"{metamodel_highres_results_output_prefix}.feature_importances.{metamodel_name}.absval_coefs.by_locus_and_model_component.png"
                                if metamodel_name != "rf_multiclass"
                                else f"{metamodel_highres_results_output_prefix}.feature_importances.{metamodel_name}.by_locus_and_model_component.png",
                            ],
                            [
                                f"{metamodel_output_prefix}.confusion_matrix.test_set_performance.{metamodel_name}.png",
                                # This one is only available for "disease":
                                f"{metamodel_highres_results_output_prefix}.confusion_matrix.test_set_performance.{metamodel_name}.expanded_confusion_matrix_disease_subtype.png",
                                f"{metamodel_highres_results_output_prefix}.confusion_matrix.test_set_performance.{metamodel_name}.expanded_confusion_matrix_ethnicity_condensed.png",
                                f"{metamodel_highres_results_output_prefix}.confusion_matrix.test_set_performance.{metamodel_name}.expanded_confusion_matrix_age_group_pediatric.png",
                            ],
                        ],
                        headers=[f"metamodel {metamodel_name}"],
                        max_width=400,
                    )
                    display(Markdown("---"))
    except FileNotFoundError as err:
        logger.warning(f"Not run: {err}")

# Overall summary

In [4]:
for target_obs_column in config.classification_targets:
    # Single-loci:
    for gene_locus in config.gene_loci_used:
        summary(
            gene_locus=gene_locus,
            target_obs_column=target_obs_column,
            show_base=True,
            show_metamodel=True,
        )

    # Multi-loci:
    if len(config.gene_loci_used) > 1:
        summary(
            gene_locus=config.gene_loci_used,
            target_obs_column=target_obs_column,
            show_base=False,
            show_metamodel=True,
        )

### GeneLocus.BCR, TargetObsColumnEnum.disease, SampleWeightStrategy.ISOTYPE_USAGE

model1 lasso_multiclass,model2 lasso_multiclass,model3-rollup lasso_multiclass
Per-fold scores: ROC-AUC (weighted OvO): 0.939 +/- 0.014 (in 3 folds) ROC-AUC (macro OvO): 0.942 +/- 0.016 (in 3 folds) au-PRC (weighted OvO): 0.938 +/- 0.015 (in 3 folds) au-PRC (macro OvO): 0.943 +/- 0.015 (in 3 folds) Accuracy: 0.812 +/- 0.010 (in 3 folds) MCC: 0.728 +/- 0.015 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.812 MCC: 0.728 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.88 0.83 0.85 63 HIV 0.82 0.89 0.85 98 Healthy/Background 0.83 0.82 0.83 221 Lupus 0.71 0.70 0.71 98 accuracy 0.81 480 macro avg 0.81 0.81 0.81 480 weighted avg 0.81 0.81 0.81 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.926 +/- 0.009 (in 3 folds) ROC-AUC (macro OvO): 0.933 +/- 0.010 (in 3 folds) au-PRC (weighted OvO): 0.927 +/- 0.014 (in 3 folds) au-PRC (macro OvO): 0.935 +/- 0.014 (in 3 folds) Accuracy: 0.761 +/- 0.010 (in 3 folds) MCC: 0.650 +/- 0.022 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.761 MCC: 0.649 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.744 +/- 0.031 (in 3 folds) MCC: 0.630 +/- 0.046 (in 3 folds) Unknown/abstention proportion: 0.034 +/- 0.030 (in 2 folds) ROC-AUC (weighted OvO): 0.931 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.940 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.935 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.945 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.744 MCC: 0.628 Unknown/abstention proportion: 0.023 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.79 0.76 0.77 63 HIV 0.85 0.83 0.84 98 Healthy/Background 0.78 0.80 0.79 221 Lupus 0.60 0.53 0.57 98 Unknown 0.00 0.00 0.00 0 accuracy 0.74 480 macro avg 0.60 0.58 0.59 480 weighted avg 0.76 0.74 0.75 480,Per-fold scores: ROC-AUC (weighted OvO): 0.829 +/- 0.032 (in 3 folds) ROC-AUC (macro OvO): 0.833 +/- 0.034 (in 3 folds) au-PRC (weighted OvO): 0.835 +/- 0.026 (in 3 folds) au-PRC (macro OvO): 0.846 +/- 0.026 (in 3 folds) Accuracy: 0.687 +/- 0.042 (in 3 folds) MCC: 0.556 +/- 0.067 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.688 MCC: 0.557 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.85 0.65 0.74 63 HIV 0.55 0.92 0.69 98 Healthy/Background 0.73 0.72 0.73 221 Lupus 0.78 0.41 0.54 98 accuracy 0.69 480 macro avg 0.73 0.67 0.67 480 weighted avg 0.72 0.69 0.68 480
,,


---

#### Metamodel flavor default

---

...

---

---

---

---

---